In [85]:
#Cat and Dog classification.

from keras.preprocessing.image import ImageDataGenerator
image_gen=ImageDataGenerator(rotation_range=30,width_shift_range=0.1,
            height_shift_range=0.1,rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,
            fill_mode='nearest')

In [86]:
#Building the model.
from keras.models import Sequential,Model
from keras.layers import ZeroPadding2D,Conv2D,BatchNormalization,DepthwiseConv2D,Input,Activation,GlobalAveragePooling2D,Reshape,Dropout
from keras import backend as K
from keras import layers
from keras.activations import relu
relu_advanced = lambda x: relu(x, max_value=6.)

def normal_conv_block(inputs,filters,alpha,kernel=(3,3),strides=(1,1)):
    filters=int(filters*alpha)
    x=ZeroPadding2D(padding=((0,1),(0,1)),name='conv1_pad')(inputs)
    x=Conv2D(filters,kernel,padding='valid',use_bias=False,strides=strides,name='conv1')(x)
    x=BatchNormalization(axis=1 if K.image_data_format() == 'channels_first' else -1,name='conv1_bn')(x)
    return Activation(relu_advanced)(x)
def depth_wise_conv_block(inputs, pointwise_conv_filters, alpha,depth_multiplier=1, strides=(1, 1), block_id=1):
    pointwise_conv_filters = int(pointwise_conv_filters * alpha)

    if strides == (1, 1):
        x = inputs
    else:
        x =ZeroPadding2D(((0, 1), (0, 1)),name='conv_pad_%d' % block_id)(inputs)
    x = DepthwiseConv2D((3, 3),
                               padding='same' if strides == (1, 1) else 'valid',
                               depth_multiplier=depth_multiplier,
                               strides=strides,
                               use_bias=False,
                               name='conv_dw_%d' % block_id)(x)
    x = BatchNormalization(axis=1 if K.image_data_format() == 'channels_first' else -1, name='conv_dw_%d_bn' % block_id)(x)
        
    x = Activation(relu_advanced)(x)

    x = Conv2D(pointwise_conv_filters, (1, 1),
                      padding='same',
                      use_bias=False,
                      strides=(1, 1),
                      name='conv_pw_%d' % block_id)(x)
    x = BatchNormalization(axis=1 if K.image_data_format() == 'channels_first' else -1,
                                  name='conv_pw_%d_bn' % block_id)(x)
    return Activation(relu_advanced)(x)
    
    
#Main model
def get_model(input_shape=(150,150,3),
              alpha=1.0,
              depth_multiplier=1,
              dropout=1e-3,
              include_top=True,
              input_tensor=None,
              pooling=None,
              classes=2,
              ):
    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    
    
    x = normal_conv_block(img_input, 32, alpha, strides=(2, 2))
    x = depth_wise_conv_block(x, 64, alpha, depth_multiplier, block_id=1)

    x = depth_wise_conv_block(x, 128, alpha, depth_multiplier,strides=(2, 2), block_id=2)
    x = depth_wise_conv_block(x, 128, alpha, depth_multiplier, block_id=3)

    x = depth_wise_conv_block(x, 256, alpha, depth_multiplier,strides=(2, 2), block_id=4)
    x = depth_wise_conv_block(x, 256, alpha, depth_multiplier, block_id=5)

    x = depth_wise_conv_block(x, 512, alpha, depth_multiplier, strides=(2, 2), block_id=6)
    x = depth_wise_conv_block(x, 512, alpha, depth_multiplier, block_id=7)
    x = depth_wise_conv_block(x, 512, alpha, depth_multiplier, block_id=8)
    x = depth_wise_conv_block(x, 512, alpha, depth_multiplier, block_id=9)
    x = depth_wise_conv_block(x, 512, alpha, depth_multiplier, block_id=10)
    x = depth_wise_conv_block(x, 512, alpha, depth_multiplier, block_id=11)

    x = depth_wise_conv_block(x, 1024, alpha, depth_multiplier,strides=(2, 2), block_id=12)
    x = depth_wise_conv_block(x, 1024, alpha, depth_multiplier, block_id=13)
    
    if include_top:
        if K.image_data_format() == 'channels_first':
            shape = (int(1024 * alpha), 1, 1)
        else:
            shape = (1, 1, int(1024 * alpha))

        x = GlobalAveragePooling2D()(x)
        x = Reshape(shape, name='reshape_1')(x)
        x = Dropout(dropout, name='dropout')(x)
        x = Conv2D(classes, (1, 1),
                          padding='same',
                          name='conv_preds')(x)
        x = Reshape((classes,), name='reshape_2')(x)
        x = Activation('softmax', name='act_softmax')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
            
    return Model(img_input,x)
            
    
    

    


In [87]:
model=get_model()
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 151, 151, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 75, 75, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 75, 75, 32)        128       
_________________________________________________________________
activation_232 (Activation)  (None, 75, 75, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 75, 75, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 75, 75, 32)        128       
__________

In [88]:
batch_size=16

train_img_gen=image_gen.flow_from_directory('E:/CATS_DOGS/CATS_DOGS/train',target_size=(150,150),batch_size=batch_size,class_mode='categorical')

Found 18743 images belonging to 2 classes.


In [89]:
test_img_gen=image_gen.flow_from_directory('E:/CATS_DOGS/CATS_DOGS/test',target_size=(150,150),batch_size=batch_size,class_mode='categorical')

Found 6251 images belonging to 2 classes.


In [90]:
model.fit_generator(train_img_gen,epochs=100,steps_per_epoch=150,validation_data=test_img_gen,validation_steps=12)

Epoch 1/100
150/150 [==============================] - 101s 671ms/step - loss: 0.8340 - acc: 0.5292 - val_loss: 1.3715 - val_acc: 0.5417
Epoch 2/100
 90/150 [=================>............] - ETA: 22s - loss: 0.7098 - acc: 0.5736

C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80000 bytes but only got 0. Skipping tag 64640
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 307363840 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 307888128 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 328728576 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensor

150/150 [==============================] - 60s 401ms/step - loss: 0.6998 - acc: 0.5829 - val_loss: 1.0886 - val_acc: 0.4688
Epoch 3/100
 85/150 [================>.............] - ETA: 25s - loss: 0.7005 - acc: 0.5868

C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262146 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262151 bytes but only got 0. Skipping tag 56
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 0. 
  warnings.warn(str(msg))


150/150 [==============================] - 62s 415ms/step - loss: 0.6891 - acc: 0.5871 - val_loss: 0.7900 - val_acc: 0.5312
Epoch 4/100
141/150 [===========================>..] - ETA: 3s - loss: 0.6760 - acc: 0.6006

...

C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6553600 bytes but only got 0. Skipping tag 49
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1050744 bytes but only got 4951. Skipping tag 51
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 293339136 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 293863424 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tenso

143/150 [===========================>..] - ETA: 2s - loss: 0.6190 - acc: 0.6617

C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 404094976 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 404619264 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 425459712 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385474 bytes but only got 6833. Skipping tag 513
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\ADMIN\Anaconda3\envs\ten

150/150 [==============================] - 57s 381ms/step - loss: 0.6222 - acc: 0.6592 - val_loss: 0.7341 - val_acc: 0.5833
Epoch 8/100
150/150 [==============================] - 57s 381ms/step - loss: 0.6079 - acc: 0.6783 - val_loss: 0.6836 - val_acc: 0.6406
Epoch 9/100
150/150 [==============================] - 57s 379ms/step - loss: 0.5975 - acc: 0.6800 - val_loss: 0.6307 - val_acc: 0.6927
Epoch 10/100
150/150 [==============================] - 57s 379ms/step - loss: 0.5816 - acc: 0.6942 - val_loss: 0.6716 - val_acc: 0.6667
Epoch 11/100
150/150 [==============================] - 57s 379ms/step - loss: 0.5779 - acc: 0.6963 - val_loss: 0.6696 - val_acc: 0.6406
Epoch 12/100
150/150 [==============================] - 58s 385ms/step - loss: 0.5602 - acc: 0.7096 - val_loss: 0.9395 - val_acc: 0.5573
Epoch 13/100
150/150 [==============================] - 59s 395ms/step - loss: 0.5456 - acc: 0.7250 - val_loss: 1.0747 - val_acc: 0.6354
Epoch 14/100
150/150 [==============================] - 

Epoch 67/100
150/150 [==============================] - 57s 382ms/step - loss: 0.2600 - acc: 0.8938 - val_loss: 0.3883 - val_acc: 0.8125
Epoch 68/100
150/150 [==============================] - 57s 381ms/step - loss: 0.2648 - acc: 0.8879 - val_loss: 0.3265 - val_acc: 0.8646
Epoch 69/100
150/150 [==============================] - 57s 380ms/step - loss: 0.2708 - acc: 0.8829 - val_loss: 0.3016 - val_acc: 0.8958
Epoch 70/100
150/150 [==============================] - 57s 381ms/step - loss: 0.2464 - acc: 0.8942 - val_loss: 0.2489 - val_acc: 0.8854
Epoch 71/100
150/150 [==============================] - 57s 380ms/step - loss: 0.2502 - acc: 0.9025 - val_loss: 0.3670 - val_acc: 0.8646
Epoch 72/100
150/150 [==============================] - 57s 380ms/step - loss: 0.2475 - acc: 0.8958 - val_loss: 1.2104 - val_acc: 0.6562
Epoch 73/100
150/150 [==============================] - 57s 379ms/step - loss: 0.2517 - acc: 0.8892 - val_loss: 0.1921 - val_acc: 0.9323
Epoch 74/100
150/150 [===================

In [93]:
model.save_weights('Dog_cat_mobilenet.h5')